# Chapter 10 - Turbo-charge your apps with advanced callback options

* Understanding State 
* Creating components that control other components 
* Allowing users to add dynamic components to the app 
* Introducing pattern-matching callbacks 

In [1]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import callback_context
import jupyter_dash as jd
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input, State, ALL, ALLSMALLER, MATCH
from dash.exceptions import PreventUpdate
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from dash_table import DataTable
import pandas as pd
import numpy as np
pd.options.display.max_columns = None

for p in [plotly, dash, jd, dcc, html, dbc, pd, np]:
    print(f'{p.__name__:-<30}v{p.__version__}')
    

plotly------------------------v4.14.3
dash--------------------------v1.20.0
jupyter_dash------------------v0.4.0
dash_core_components----------v1.16.0
dash_html_components----------v1.1.3
dash_bootstrap_components-----v0.12.0
pandas------------------------v1.2.3
numpy-------------------------v1.20.1


In [2]:
poverty = pd.read_csv('../data/poverty.csv', low_memory=False)
poverty.head(1)

,Country Name,Country Code,year,"Annualized growth in per capita real survey mean consumption or income, bottom 40% (%)","Annualized growth in per capita real survey mean consumption or income, top 10% (%)","Annualized growth in per capita real survey mean consumption or income, top 60% (%)","Annualized growth in per capita real survey mean consumption or income, total population (%)",Annualized growth in per capita real survey median income or consumption expenditure (%),GINI index (World Bank estimate),Growth component of change in poverty at $1.90 a day (2011 PPP) (% of change),Growth component of change in poverty at $3.20 a day (2011 PPP) (% of change),Growth component of change in poverty at $5.50 a day (2011 PPP) (% of change),Income share held by fourth 20%,Income share held by highest 10%,Income share held by highest 20%,Income share held by lowest 10%,Income share held by lowest 20%,Income share held by second 20%,Income share held by third 20%,Median daily per capita income or consumption expenditure (2011 PPP),"Multidimensional poverty, Drinking water (% of population deprived)","Multidimensional poverty, Educational attainment (% of population deprived)","Multidimensional poverty, Educational enrollment (% of population deprived)","Multidimensional poverty, Electricity (% of population deprived)","Multidimensional poverty, Headcount ratio (% of population)","Multidimensional poverty, Monetary poverty (% of population deprived)","Multidimensional poverty, Sanitation (% of population deprived)",Number of poor at $1.90 a day (2011 PPP) (millions),Number of poor at $3.20 a day (2011 PPP) (millions),Number of poor at $5.50 a day (2011 PPP) (millions),"Population, total",Poverty gap at $1.90 a day (2011 PPP) (%),Poverty gap at $3.20 a day (2011 PPP) (% of population),Poverty gap at $5.50 a day (2011 PPP) (% of population),Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population),"Poverty headcount ratio at $1.90 a day, Female (2011 PPP) (% of female population)","Poverty headcount ratio at $1.90 a day, Male (2011 PPP) (% of male population)","Poverty headcount ratio at $1.90 a day, age 0-14 (2011 PPP) (% of population age 0-14)","Poverty headcount ratio at $1.90 a day, age 15-64 (2011 PPP) (% of population age 15-64)","Poverty headcount ratio at $1.90 a day, age 65+ (2011 PPP) (% of population age 65+)","Poverty headcount ratio at $1.90 a day, rural (2011 PPP) (% of rural population)","Poverty headcount ratio at $1.90 a day, urban (2011 PPP) (% of urban population)","Poverty headcount ratio at $1.90 a day, with primary education (2011 PPP) (% of population age 16+ with primary education)","Poverty headcount ratio at $1.90 a day, with secondary education (2011 PPP) (% of population age 16+ with secondary education)","Poverty headcount ratio at $1.90 a day, without education (2011 PPP) (% of population age 16+ without education)","Poverty headcount ratio at $1.90 a day, with Tertiary/post-secondary education (2011 PPP) (% of population age 16+ with Tertiary/post-secondary education)",Poverty headcount ratio at $3.20 a day (2011 PPP) (% of population),Poverty headcount ratio at $5.50 a day (2011 PPP) (% of population),Poverty headcount ratio at national poverty lines (% of population),"Poverty headcount ratio at national poverty lines (% of population), including noncomparable values",Redistribution component of change in poverty at $1.90 a day (2011 PPP) (% of change),Redistribution component of change in poverty at $3.20 a day (2011 PPP) (% of change),Redistribution component of change in poverty at $5.50 a day (2011 PPP) (% of change),"Survey mean consumption or income per capita, bottom 40% (2011 PPP $ per day)","Survey mean consumption or income per capita, top 10% (2011 PPP $ per day)","Survey mean consumption or income per capita, top 60% (2011 PPP $ per day)","Survey mean consumption or income per capita, total population (2011 PPP $ per day)",Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Spe

In [3]:
import time

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    html.Br(), html.Br(),
    dcc.Dropdown(id='dropdown', options=[{'label': x, 'value': x}
                                         for x in ['one', 'two', 'three']]),
    html.Br(),
    dcc.Textarea(id='textarea', cols=50, rows=5),
    html.Br(),
    html.Div(id='output'),
    
])


@app.callback(Output('output', 'children'), 
              Input('dropdown', 'value'),
              Input('textarea', 'value'))
def display_values(dropdown_val, textarea_val):
    time.sleep(4)
    return f'You chose "{dropdown_val}" from the dropdown, and wrote "{textarea_val}" in the textarea.'

app.run_server(mode='inline', port=8050, height=400)

In [4]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    html.Br(), html.Br(),
    dcc.Dropdown(id='dropdown', options=[{'label': x, 'value': x}
                                         for x in ['one', 'two', 'three']]),
    html.Br(),
    dcc.Textarea(id='textarea', cols=50, rows=5),
    html.Br(),
    dbc.Button("Submit", id='button'),
    html.Br(), html.Br(),
    html.Div(id='output')
])


@app.callback(Output('output', 'children'), 
              Input('button', 'n_clicks'),
              State('dropdown', 'value'),
              State('textarea', 'value'))
def display_values(n_clicks, dropdown_val, textarea_val):
    if not n_clicks:
        raise PreventUpdate
    return f'You chose "{dropdown_val}" from the dropdown, and wrote "{textarea_val}" in the textarea.'

app.run_server(mode='inline', port=8051, height=300)

In [5]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    html.Br(),html.Br(),
    dbc.Button("Add Chart", id='button'),
    html.Div(id='output', children=[])
])

@app.callback(Output('output', 'children'),
              Input('button', 'n_clicks'),
              State('output', 'children'))
def add_new_chart(n_clicks, children):
    if not n_clicks:
        raise PreventUpdate
    new_chart = dcc.Graph(figure=px.bar(height=300, width=500, title=f"Chart {n_clicks}"))
    
    children.append(new_chart)
    return children

app.run_server(port=8052)

Dash app running on http://127.0.0.1:8052/


In [6]:
countries = poverty[poverty['is_country']]['Country Name'].drop_duplicates().sort_values()

In [7]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    html.Br(),html.Br(),
    dbc.Row([
        dbc.Col(lg=1),
        dbc.Col([
            dbc.Button("Add Chart", id='button'),

            html.Div(id='output', children=[])
            
        ], lg=4)
    ]),
])

@app.callback(Output('output', 'children'),
              Input('button', 'n_clicks'),
              State('output', 'children'))
def add_new_chart(n_clicks, children):
    if not n_clicks:
        raise PreventUpdate
    new_chart = dcc.Graph(id={'type': 'chart', 'index': n_clicks}, 
                          figure=px.bar(height=300, width=500,
                                        title=f"Chart {n_clicks}"))
    
    new_dropdown = dcc.Dropdown(id={'type': 'dropdown', 'index': n_clicks},
                                options=[{'label': c, 'value': c}
                                         for c in poverty[poverty['is_country']]['Country Name'].drop_duplicates().sort_values()])
    
    children.append(html.Div([
        new_chart, new_dropdown
    ]))
    return children

@app.callback(Output({'type': 'chart', 'index': MATCH}, 'figure'), 
              Input({'type': 'dropdown', 'index': MATCH}, 'value'))
def create_population_chart(country):
    if not country:
        raise PreventUpdate
    df = poverty[poverty['Country Name']==country]
    fig = px.line(df, x='year', y='Population, total', title=f'Population of {country}')
    return fig


app.run_server(port=8053)

Dash app running on http://127.0.0.1:8053/


In [8]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    html.Br(),
    html.Div(id='feedback'),
    dbc.Label("Create your own dropdown, add options one per line:"),
    dbc.Textarea(id='text', cols=40, rows=5),
    html.Br(),
    dbc.Button("Set options", id='button'),
    html.Br(),
    dcc.Dropdown(id='dropdown'),
    dcc.Graph(id='chart')
])


@app.callback(Output('dropdown', 'options'),
              Output('feedback', 'children'),
              Input('button', 'n_clicks'),
              State('text', 'value'))
def set_dropdown_options(n_clicks, options):
    if not n_clicks:
        raise PreventUpdate
    text = options.split()
    message = dbc.Alert(f"Success! you added the options: {', '.join(text)}", 
                        color='success',
                        dismissable=True)
    options = [{'label': t, 'value': t} for t in text]
    return options, message

@app.callback(Output('chart', 'figure'),
              Input('dropdown', 'value'))
def create_population_chart(country_code):
    if not country_code:
        raise PreventUpdate
    df = poverty[poverty['Country Code']==country_code]
    return px.line(df, x='year', y='Population, total', title=f"Population of {country_code}")

app.run_server(height=1500, port=8054)

Dash app running on http://127.0.0.1:8054/


In [9]:
poverty['Country Code'].unique()

array(['AFG', 'ALB', 'DZA', 'AGO', 'ARG', 'ARM', 'AUS', 'AUT', 'AZE',
       'BGD', 'BLR', 'BEL', 'BLZ', 'BEN', 'BTN', 'BOL', 'BIH', 'BWA',
       'BRA', 'BGR', 'BFA', 'BDI', 'CPV', 'KHM', 'CMR', 'CAN', 'CAF',
       'TCD', 'CHL', 'CHN', 'COL', 'COM', 'COD', 'COG', 'CRI', 'CIV',
       'HRV', 'CYP', 'CZE', 'DNK', 'DJI', 'DOM', 'EAS', 'ECU', 'EGY',
       'SLV', 'GNQ', 'ERI', 'EST', 'SWZ', 'ETH', 'ECS', 'FJI', 'FIN',
       'FCS', 'FRA', 'GAB', 'GMB', 'GEO', 'DEU', 'GHA', 'GRC', 'GTM',
       'GIN', 'GNB', 'GUY', 'HTI', 'HIC', 'HND', 'HUN', 'DFS', 'IDA',
       'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR', 'ITA', 'JAM',
       'JPN', 'JOR', 'KAZ', 'KEN', 'KIR', 'KOR', 'XKX', 'KGZ', 'LAO',
       'LCN', 'LVA', 'LBN', 'LSO', 'LBR', 'LTU', 'LMY', 'LIC', 'LMC',
       'LUX', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MRT', 'MUS',
       'MEX', 'FSM', 'MEA', 'MIC', 'MDA', 'MNG', 'MNE', 'MAR', 'MOZ',
       'MMR', 'NAM', 'NPL', 'NLD', 'NIC', 'NER', 'NGA', 'MKD', 'NOR',
       'PAK', 'PLW',

In [10]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

df = poverty[poverty['is_country']]
import time
app.layout = html.Div([
    html.Br(),
    dbc.Row([
        dbc.Col(lg=1),
        dbc.Col([
            dbc.Label('Select the year:'),
            dcc.Slider(id='year_cluster_slider',
                       min=1974, max=2018, step=1, included=False,
                       value=2018,
                       marks={year: str(year)
                              for year in range(1974, 2019, 5)})
        ], lg=6, md=12),
        dbc.Col([
            dbc.Label('Select the number of clusters:'),
            dcc.Slider(id='ncluster_cluster_slider',
                       min=2, max=15, step=1, included=False,
                       value=4,
                       marks={n: str(n) for n in range(2, 16)}),
        ], lg=4, md=12)
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col(lg=1),
        dbc.Col([
            dbc.Label('Select Indicators:'),
            dcc.Dropdown(id='cluster_indicator_dropdown',optionHeight=40,
                         multi=True,
                         value=['GINI index (World Bank estimate)'],
                         options=[{'label': indicator, 'value': indicator}
                                  for indicator in poverty.columns[3:54]]),
        ], lg=6),
        dbc.Col([            
            dbc.Label(''),html.Br(),
            dbc.Button("Submit", id='clustering_submit_button'),
        ]),
    ]),
    dcc.Loading([
                dcc.Graph(id='clustered_map_chart')
    ])
], style={'backgroundColor': '#E5ECF6'})

@app.callback(Output('clustered_map_chart', 'figure'),
              Input('clustering_submit_button', 'n_clicks'),
              State('year_cluster_slider', 'value'),
              State('ncluster_cluster_slider', 'value'),
              State('cluster_indicator_dropdown', 'value'))
def clustered_map(n_clicks, year, n_clusters, indicators):
    if not indicators:
        raise PreventUpdate
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    scaler = StandardScaler()
    kmeans = KMeans(n_clusters=n_clusters)
    
    df = poverty[poverty['is_country'] & poverty['year'].eq(year)][indicators + ['Country Name', 'year']]
    data = df[indicators]
    if df.isna().all().any():
        return px.scatter(title='No available data for the selected combination of year/indicators.')
    data_no_na = imp.fit_transform(data)
    scaled_data = scaler.fit_transform(data_no_na)
    kmeans.fit(scaled_data)

    fig = px.choropleth(df,
                        locations='Country Name',
                        locationmode='country names',
                        color=[str(x) for x in  kmeans.labels_], 
                        labels={'color': 'Cluster'},
                        hover_data=indicators,
                        height=650,
                        title=f'Country clusters - {year}. Number of clusters: {n_clusters}<br>Inertia: {kmeans.inertia_:,.2f}',
                        color_discrete_sequence=px.colors.qualitative.T10)
    fig.add_annotation(x=-0.1, y=-0.15, 
                       xref='paper', yref='paper',
                       text='Indicators:<br>' + "<br>".join(indicators), 
                       showarrow=False)
    fig.layout.geo.showframe = False
    fig.layout.geo.showcountries = True
    fig.layout.geo.projection.type = 'natural earth'
    fig.layout.geo.lataxis.range = [-53, 76]
    fig.layout.geo.lonaxis.range = [-137, 168]
    fig.layout.geo.landcolor = 'white'
    fig.layout.geo.bgcolor = '#E5ECF6'
    fig.layout.paper_bgcolor = '#E5ECF6'
    fig.layout.geo.countrycolor = 'gray'
    fig.layout.geo.coastlinecolor = 'gray'
    return fig
    
app.run_server(height=1200, debug=True, mode='inline', port=8055)